In [1]:
import ast
import imdb
import json
import pickle
import itertools
import dateparser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ast import literal_eval
from price_parser import Price
from collections import Counter

%matplotlib inline

#### Step 1: Preprocessing, data scraping

In [2]:
DATA_COLUMNS = ["index","wikipedia_id", "imdb_id", "title", "cast", "genres", "runtimes", "countries", "languages", "box_office", "rating", "votes", "plot", "synopsis"]

df = pd.read_csv("./Data/fetched_data.csv", names=DATA_COLUMNS, header=0, index_col=0)

In [3]:
df["title"] = df["title"].astype("string")
df.cast = df.cast.apply(lambda x: literal_eval(x))
df.genres = df.genres.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df["runtimes"] = pd.to_numeric(df["runtimes"].apply(lambda x: "0" if pd.isna(x) else x.replace("[", "").replace("]", "").replace("'", "")))
df.countries = df.countries.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df.languages = df.languages.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df["synopsis"] = df["synopsis"].apply(lambda x: "" if pd.isna(x) else literal_eval(x)[0])
df["plot"] = df["plot"].apply(lambda x: "" if pd.isna(x) else literal_eval(x)[0])

In [4]:
# TODO question assistant => comment avoir la bonne valeur de box office en dollar 
# sachant que le cours a evolue au cours du temps

def convert_currency(x):
    formated = Price.fromstring(x)
    amount = formated.amount_float
    match formated.currency:
        case "$":
            return amount
        case "EUR":
            return amount * 1.18
        case "GBP":
            return amount * 1.36
        case "SEK":
            return amount * 0.11
        case "CAD":
            return amount * 0.79
        case "INR":
            return amount * 0.013
        case "CZK":
            return amount * 0.04
        case "JPY":
            return amount * 0.009
        case _:
            return None
    

In [5]:
# lets format the box office column

def superfunction(box_office):
    if pd.isna(box_office):
        return None, None, None
    film = ast.literal_eval(box_office)
    budget = convert_currency(film["Budget"]) if "Budget" in film else None
    bo_usa = Price.fromstring(film["Opening Weekend United States"]).amount if "Opening Weekend United States" in film else None
    bo_world = Price.fromstring(film["Cumulative Worldwide Gross"]).amount if "Cumulative Worldwide Gross" in film else None
    return budget, bo_usa, bo_world
    
formated_bo = df["box_office"].apply(lambda x: superfunction(x))
df["budget"], df["box_office_usa"], df["box_office_world"] = zip(*formated_bo)


In [6]:
years_df = pd.read_csv("./Data/movie.metadata.tsv", sep='\t', header=None)

In [7]:
# for each movies we have in df, find by id the one in years df
def parse_date(x):
    timestamped_fetched = years_df[years_df[0] == x][3].values
    year = dateparser.parse(timestamped_fetched[0]).year
    return year

df["release_year"] = df.wikipedia_id.apply(lambda x: parse_date(x))

In [ ]:
df.to_csv("./Data/cleaned_data.csv")

#### Step 2: Investigating the evolution of movie production over time and finding main character

In [ ]:
# 2.1 movie production

In [ ]:
# main character


#### Step 3: Box office feature analysis

#### Step 4: Finding correlation

#### Step 5: Predicting future production and box office

#### Step 6: Final visualisation of the datastory